In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import numpy as np
import io
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [4]:
sitemaps = parse_sitemap(io.StringIO(req.get('https://russian.rt.com/sitemap.xml').content.decode()))
sitemaps_filtered = []
for sitemap in sitemaps:
    if re.search(r'2019|2020', sitemap['url']):
        sitemaps_filtered.append(sitemap)

In [5]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemaps_filtered):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-5-73c316e576b5>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemaps_filtered):


In [8]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True)
df.head()

,url,lastmod
0,https://russian.rt.com/tag/alberto-fernandes,2019-10-29 07:52:02+00:00
1,https://russian.rt.com/tag/aleksandr-bessmertnyh,2019-11-29 13:31:27+00:00
2,https://russian.rt.com/tag/aleksandr-tretyakov,2019-11-29 13:35:16+00:00
3,https://russian.rt.com/tag/aleksandra-boikova-...,2019-11-29 13:48:44+00:00
4,https://russian.rt.com/tag/aleksandra-trusova,2019-09-26 12:17:33+00:00


In [9]:
df[(df.lastmod >= '2019-11-01')].to_json('news_russia_today.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape
#df[(df.lastmod >= '2019-11-01') & (df.lastmod < '2019-07-01')].shape

(117247, 2)

# Parsing

In [2]:
df = pd.read_json('db/news_rt_full_db.json')
df.head()

,lastmod,url
0,2018-01-06T10:05:00Z,https://russian.rt.com/sport/article/467618-sb...
1,2018-01-15T05:06:00Z,https://russian.rt.com/foto/350493-santa-barba...
2,2018-12-01T08:15:00Z,https://russian.rt.com/world/article/578178-pr...
3,2018-02-20T05:48:00Z,https://russian.rt.com/world/foto/361090-churk...
4,2018-10-26T07:09:00Z,https://russian.rt.com/nopolitics/foto/567698-...


In [8]:
def parse_rt_news(text):
    soup = BeautifulSoup(text)
    news_text_list_first = soup.find_all(name="div", attrs={'class': "arcticle-content"})
    news_text_list_second = soup.find_all(name="div", attrs={'class': "js-mediator-article"})
    news_text_list = news_text_list_first if news_text_list_first else (news_text_list_second if news_text_list_second else '')
    news_text = ''
    
    if news_text_list:
        for news in news_text_list:
            bad = news.find_all(name=['script', 'iframe', 'svg', 'figure'])
            if bad:
                [tag.decompose() for tag in bad]
        
            news_text += " " + news.get_text(" ", strip=True)
        news_text = re.sub(r'[\n\r]+', ' ', news_text).replace(u'\xa0', u' ')

    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+', ' ', description_elem.get('content')).replace(u'\xa0', u' ') if description_elem else ''
    
    return news_text, description

In [9]:
parse_rt_news(req.get('https://russian.rt.com/world/news/529049-pompeo-rossiya-ssha').text)

(' Госсекретарь США Майк Помпео в интервью The Wall Street Journal заявил, что Соединённые Штаты должны проявлять гибкость в отношениях с Россией. По его словам, «существует много вещей», в которых США не согласны с Россией. «У нас невероятно разные ценности, но есть и области, где мы находим совпадения», — приводит его слова РИА Новости . Помпео добавил, что задача госсекретаря Соединённых Штатов заключается «в определении тех областей», в которых страны могут работать вместе, «в то же время защищая США от худших действий». Ранее глава российского внешнеполитического ведомства Сергей Лавров заявил, что Россия готова к контактам с США.',
 'Помпео рассказал о необходимости проявлять гибкость в отношениях с Россией - Читайте подробнее на сайте РТ на русском.')